In [0]:
# READ JSON
df1 = spark.read.json(path= "dbfs:/FileStore/shared_uploads/tacnampt@gmail.com/airports-3.json")
df1.show(2)
df1.printSchema()

+--------------------+--------------------+--------------------+--------------------+
|          airport_id|                city|                name|               state|
+--------------------+--------------------+--------------------+--------------------+
|{10165, 10299, 12...|{Adak Island, Anc...|{Adak, Ted Steven...|{AK, AK, AK, FL, ...|
+--------------------+--------------------+--------------------+--------------------+

root
 |-- airport_id: struct (nullable = true)
 |    |-- 0: long (nullable = true)
 |    |-- 1: long (nullable = true)
 |    |-- 10: long (nullable = true)
 |    |-- 100: long (nullable = true)
 |    |-- 101: long (nullable = true)
 |    |-- 102: long (nullable = true)
 |    |-- 103: long (nullable = true)
 |    |-- 104: long (nullable = true)
 |    |-- 105: long (nullable = true)
 |    |-- 106: long (nullable = true)
 |    |-- 107: long (nullable = true)
 |    |-- 108: long (nullable = true)
 |    |-- 109: long (nullable = true)
 |    |-- 11: long (nullable = tr

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# CREANDO UN ESQUEMA PARA EL JSON
schema = StructType() \
             .add("airport_id", MapType(StringType(), IntegerType())) \
             .add("city", MapType(StringType(), StringType())) \
             .add("name", MapType(StringType(), StringType())) \
             .add("state", MapType(StringType(), StringType()))

# READ JSON USANDO EL ESQUEMA
df1 = spark.read.json(path= "dbfs:/FileStore/shared_uploads/tacnampt@gmail.com/airports-3.json", schema=schema)
df1.show(2)
df1.printSchema()


+--------------------+--------------------+--------------------+--------------------+
|          airport_id|                city|                name|               state|
+--------------------+--------------------+--------------------+--------------------+
|{0 -> 10165, 1 ->...|{0 -> Adak Island...|{0 -> Adak, 1 -> ...|{0 -> AK, 1 -> AK...|
+--------------------+--------------------+--------------------+--------------------+

root
 |-- airport_id: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)
 |-- city: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- name: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- state: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [0]:
# CON EXPLODE CREAMOS CUATRO DATAFRAMES, PARA CONVERTIR A FORMATO TABULAR
df1_a = df1.select(explode("airport_id").alias("id","airport_id")) 
df1_b = df1.select(explode("city").alias("id","city"))
df1_c = df1.select(explode("name").alias("id","name"))
df1_d = df1.select(explode("state").alias("id","state"))


In [0]:
# AHORA HACEMOS UN JOIN DE LOS DATAFRAMES
df_final = df1_a \
    .join(df1_b, df1_a.id == df1_b.id, "inner") \
    .join(df1_c, df1_a.id == df1_c.id, "inner") \
    .join(df1_d, df1_a.id == df1_d.id, "inner") \
    .select(df1_a.airport_id, df1_b.city, df1_c.name, df1_d.state)

df_final.show(5)

+----------+-----------+--------------------+-----+
|airport_id|       city|                name|state|
+----------+-----------+--------------------+-----+
|     10165|Adak Island|                Adak|   AK|
|     10299|  Anchorage|Ted Stevens Ancho...|   AK|
|     10304|      Aniak|       Aniak Airport|   AK|
|     10754|     Barrow|Wiley Post/Will R...|   AK|
|     10551|     Bethel|      Bethel Airport|   AK|
+----------+-----------+--------------------+-----+
only showing top 5 rows

